# Model Garden を使った物体検出

## インストール

In [ ]:
%%bash

mkdir -p .conan
cd .conan
conan install ..

In [ ]:
%%bash

export PATH=$PWD/.conan:$PATH

cd models/research
protoc object_detection/protos/*.proto --python_out=.
cp -f object_detection/packages/tf2/setup.py .
pip install .

## 定数

In [ ]:
# 適当な画像を用意する
IMAGE_PATH = "dog.jpg"

# 使用するモデルの ID を model_urls.csv から選択
MODEL_ID = 27

# 推論結果のスコアに対するしきい値
SCORE_THRESHOLD = 0.5

## インポート

In [ ]:
import csv
from pathlib import Path

import tensorflow as tf
from object_detection.utils.label_map_util import create_category_index_from_labelmap
import numpy as np
from PIL import Image, ImageDraw, ImageColor, ImageFont
import certifi

## モデルをダウンロード

In [ ]:
# ダウンロードに失敗しないようにするためのおまじない
os.environ["SSL_CERT_FILE"] = certifi.where()

In [ ]:
with open("model_urls.csv") as f:
     model_urls = list(csv.DictReader(f))

In [ ]:
model_url = model_urls[MODEL_ID]

model_dir = tf.keras.utils.get_file(
    fname=model_url["name"],
    origin=model_url["url"],
    untar=True,
    cache_subdir="models",
)
model_dir = str(Path(model_dir) / "saved_model")
model_dir

## モデルをロード

In [ ]:
model = tf.saved_model.load(model_dir)
infer = model.signatures["serving_default"]

!saved_model_cli show --all --dir $model_dir

In [ ]:
category_index = create_category_index_from_labelmap(
    "models/research/object_detection/data/mscoco_label_map.pbtxt",
    use_display_name=True,
)

## 画像データをロード

In [ ]:
image = tf.io.read_file(IMAGE_PATH)
image = tf.io.decode_jpeg(image)
image.shape

## 画像データを正規化

In [ ]:
#
# モデルによっては float32 への変換が必要かも（入力層の dtype を確認）
#
# image = tf.image.convert_image_dtype(image, tf.float32)

images = tf.stack([image], axis=0)
images.shape

## 推論を実行

In [ ]:
preds = infer(images)

num_detections = int(preds["num_detections"][0])
boxes = preds["detection_boxes"][0]
scores = preds["detection_scores"][0]
labels = preds["detection_classes"][0]

num_detections, boxes.shape, scores.shape, labels.shape

## バウンディングボックスを描画

In [ ]:
output_image = Image.open(IMAGE_PATH)

draw = ImageDraw.Draw(output_image)
color = ImageColor.getrgb("#E91E63")
font = ImageFont.truetype("Monaco", size=46)

xy_scale = np.array([output_image.height, output_image.width, output_image.height, output_image.width])

for i in range(num_detections):
    label = int(labels[i])
    score = scores[i]

    if score <= SCORE_THRESHOLD:
        continue

    name = category_index[label]["name"]
    text = f"{name}: {score * 100:.1f}%"
    y_min, x_min, y_max, x_max = boxes[i] * xy_scale

    print(f"{name},{label},{score:.6f},{x_min:.6f},{y_min:.6f},{x_max:.6f},{y_max:.6f}")

    draw.text((x_min, y_min), text, font=font, fill=color)
    draw.rectangle((x_min, y_min, x_max, y_max), outline=color, width=4)

output_image